# Introduction
ADG4 is a package which computes a value while using automatic
differentiation to compute derivatives of that value. It integrates a set of techniques
to evaluate the derivative of various mathematical expressions.

Being able to take derivatives has many important practical applications across a variety of domains.
In physics, the first order derivative tells us the rate of change, while further orders can tell us
more pieces of information like acceleration and actergy. In statistics, applications include Bayesian
inference and the training of neural networks. In economics, taking the derivative of profit and utility
functions allows agents to maximize their expected outcomes. These are only a few instances wherein 
differentiating complex functions is useful and necessary.

Given the broad set of applications across domains, we hope ADG4 can be a useful tool
that can facilitate quick and easy numerical evaluation of derivatives through computing.

# Background


The methodology behind our package relies on the fact that every function can be computed as
a combination of simple elementary functions, which have simple derivatives (exp, log, sin, cos, etc.).
By applying the chain rule repeatedly to these operations, derivatives (or more generally, 
Jacobian matries) of arbitrary functions can automatically calculated. 

The basic functions of our package are based on the forward mode of auto differentiation.
The forward mode takes the first step as putting in seeding variables at the place of each
independent variable, and then recursively move forward following the chain rule to 
combine all the sub-expressions to get the final derivatives. In other words, forward mode
involves repeatedly substituting the derivatives of the inner functions in the chain rule.
We are going to leverage dual numbers to operate the forward mode automatic differentiation, 
which are vectors containing a function value and derivative. The algebra of dual numbers is 
exactly analogous to the forward accumulation process of calculating the derivatives following
the chain rule.


As opposed to forward mode, reverse mode is another way of calculating automatic differentiation,
which can be a helpful extension of our package. Reverse method takes the first step as
calculating the derivatives of the outermost functions, and then using the chain rule to 
recursively calculate the derivatives of the inner functions, until it gets to each 
independent variables, where the seeding variables would be put in.



Here are a few important concepts pertaining AD which are mentioned above:
- The Chain Rule: the derivative of a convoluted function is the product of each simple
 function evaluated at the value of its child function.
- Jacobian: the gradient of each element of a function's output with respect to each and every input. In other words, 
it's the gradient of the function at the space spanned by the inputs.
- Dual numbers: a two dimentional space where a outer product is defined between any vectors $$x\ =\ (a,\ b)$$ and $$y\ =\ (c,\ d)$$ as $$x\times y\ =\ (a\cdot c,\ a\cdot d\ +\ c\cdot b)$$.
 Dual number is useful when we want to include a second-order calculation, 
 i.e., not only calculate the value of a function, but also its changes with regard to small changes from the inputs.











# How to use

We envision a three-step user-software interaction:
- Download and install: the package will be distributed through PyPI. A user should download it and install it along with all other needed packages automatically.
Some code like the following can be used to import the libraries:

   `import ADG4`


- Please refer to the Implementation section for more details.

- Functional inputs: A class should be called to instantiate the object. The constructor requires the following inputs: a list of function inputs as declaration, a list of input values, a function form (methods for repetition and recursion should be provided in preparation of cases like f = x1 x2 ... x100000)
- Jacobian: Lastly, the function returns to the Jacobian matrix. 

Adds-on: solution to a three-body interaction dataset. The dataset provides the trajectories of the three bodies. Function returns to the mass ratios.



# Software Organization

Directory will be structured based on functionality and modules will be deployed according to program features such as mathematic calculations, user interface, computational graph and unit tests. 
Use of third-party libraries or modules are being considered for support specific features of the project, but nothing specific has been defined yet. 

The project will leverage the unittest module to test, and will live in a separate directory structure.  
The test suite will be run automatically via TravisCI everytime we push a change into our branch.  Below is an example of unit test implementation:  

Initially we will deploy and clone repo via Git, but we intend to use PyPI and make it friendly to use on different environments such as conda, virtualenv, etc. 

In [ ]:
import unittest
from module1 import ImportantFunction
 
class TestImportantFunction(unittest.TestCase):
    """Runs tests related to the implementation of Important Function"""
    def setUp(self):
        self.func = ImportantFunction()
 
    def test_1(self):
        self.assertTrue(True)
 
    def test_2(self):
        self.assertEqual(self.func.state, 0)
 
    def test_4(self):
        self.func.increment_state()
        self.assertEqual(self.func.state, 1)
 
    def test_5(self):
        self.func.increment_state()
        self.func.increment_state()
        self.func.clear_state()
        self.assertEqual(self.func.state, 0)
 
if __name__ == '__main__':
    unittest.main()

# Implementation

The core data structure is an ADVariable which maintains two core pieces of data:
the current value, and the the current trace for *all* previously involved variables.
Eg the partial derivatives with respect to each input variable.


## Core Classes 
### ADVariable
```
ADVariable
  fields:
    - val: the currently computed value
    - derivs: a dictionary that maps inputs -> partial derivatives
  methods:
    - get_deriv(wrt=None)
    - __mul__, __add__, .. call the associated ADFunctions to return new ADVariables
```

### Managing derivatives
At any point, the method `get_deriv(wrt=v)` can be called on an ADVariable to get the derivative with respont to any input


## ADFunctions
ADFunctions accept one or more ADVariables and output a new ADVariable with an updated `val` and `derivs`.
```
ADFunction
  input: One or more ADVariables
  output: A new ADVariable with appropriate val, deriv
```



### Example
```
x = ADVariable(1)
y = ADVariable(2)
z = x + (2 * y) # behind the scenes this calls the Add ADFunction and the mul ADFunction respectively

z.get_deriv(wrt=x) # returns 1
z.get_deriv(wrt=y) # returns 2

z.derivs # contains {x: 1, y : 2}, eg dx and dy
z.val # contains 5
```


### ADFunction Implementations
Since ADFunctions do not contain internal state, for now it seems to make the most sense to have them as direct functions.
Elementary functions like sin, cos, etc will each have a corresponding `ADFunction` implementation that stores gradients
and maintains logic depending on whether the input is a vector, matrix etc. For functions such as multiplication and addition,
which have implicit functionality in Python, we will overwrite the underlying dunder method so that when called upon an `ADVariable`,
these functions will evaluate and update the derivatives as described above.

No external dependencies are required at this time.


### Vectors, Matrices
A Vector is a `list(ADVariables)` and a matrix is a `list(list(ADVariables))`.
If each `ADFunction` handles the four input types:
- Scalar
- ADVariable
- list(ADVariable)
- list(list(ADVariable))

We anticipate that our program will be able to support these instances.

####  Vectors, Matrices example:

```
v = [ ADVariable(1) for i in range(4)] # a vector of length 4
z = v + 2
```

```
Add(v1, v2)
 - if v1 is scalar -> ...
 ...
 - if v1 is vector -> ..
```
-------------


From project page of course site for reference:
Discuss how you plan on implementing the forward mode of automatic differentiation.

What are the core data structures?
What classes will you implement?
What method and name attributes will your classes have?
What external dependencies will you rely on?
How will you deal with elementary functions like sin, sqrt, log, and exp 
(and all the others)?
Be sure to consider a variety of use cases. For example, don't limit your design to 
scalar functions of scalar values. Make sure you can handle the situations of vector functions
of vectors and scalar functions of vectors. Don't forget that people will want to use your
library in algorithms like Newton's method (among others).


# Sample Implementation
Below is working code that sketches out an implementation using the above design.

In [4]:
## EXAMPLE IMPLEMENTATION SECTION IN PROGRESS
## Below denotes an example of what code structure could look like

# All functions return a new ADVariable with an updated value and list of derivatives with respect to variabls
def add(first, second):
	val = first.val + second.val
	derivs = {}

    # Update trace for the first item
	for element, element_deriv in first.derivs.items():
		derivs[element] = element_deriv

    # Update trace for the second item
	for element, element_deriv in second.derivs.items():
		derivs[element] = element_deriv

	n = ADVariable(val=val, derivs=derivs)
	return n

print('add(ADVariable, ADVariable) -> returns new ADVariable with updated derivatives')

def mul_scalar(first, scalar):
	val = first.val * scalar
	derivs = {}
	for element, element_deriv in first.derivs.items():
		derivs[element] = element_deriv * scalar

	n = ADVariable(val=val, derivs=derivs)
	return n


# ADVariables track their trace inside the derivs variable
class ADVariable:

	def __init__(self, val, derivs={}):
		self.val = val
		self.derivs = derivs
		self.derivs[self] = 1

	def get_deriv(self, wrt=None):
		if not wrt:
			return self.derivs
		return self.derivs[wrt]

	def __add__(self, other):
		return add(self, other)

	def __repr__(self):
		return f'ADVariable(val={self.val})'



x = ADVariable(4)
y = ADVariable(2)

print(x)
print(y)

z = add(x, mul_scalar(y, 2)) # x + y * 2

print(z)

b = mul_scalar(z, 4)

print(f'The derivative of b = 4(x + 2y) wrt y is {b.get_deriv(wrt=y)}')
print(f'The derivative of b = 4z We wrt z, where z= x+ 2y is {b.get_deriv(wrt=z)}')


add(ADVariable, ADVariable) -> returns new ADVariable with updated derivatives
ADVariable(val=4)
ADVariable(val=2)
ADVariable(val=8)
The derivative of b = 4(x + 2y) wrt y is 8
The derivative of b = 4z wrt z, where z= x+ 2y is 4
